# メタデータを検証する

研究データに付与されているメタデータを検証し、その結果をレポートします。  
検証を行うことで、適切なメタデータが研究データに付与されているかを確認することができます。  

検証結果は必ず同期する必要はありません。この環境で検証結果を確認いただいたあとに、GIN-forkへ同期するか破棄するか選択いただけます。  
検証結果を同期する場合は、研究リポジトリの直下の**validation_results**フォルダに格納されます。タスク実行後に検証結果を確認する際はvalidation_resultsフォルダ配下の**results.json**をご確認ください。  
同期しない場合は、検証結果は破棄されます。

## 0. 研究リポジトリ名を確認する

以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。 

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. 研究リポジトリのメタデータを用意する

メタデータを検証するための準備として、研究リポジトリのメタデータの取得と整形を行います。

In [ ]:
import os
os.chdir(os.environ['HOME'])

# 現在のブランチ名を取得する
result = !git branch --contains
current_branch = result[0].replace('* ', '')

# repository_idを取得する
from WORKFLOWS.utils.repository_id import repository_id
repo_id = repository_id.get_repo_id()

# ginforkのtokenを取得する
from WORKFLOWS.utils.token import token
ginfork_token = token.get_ginfork_token()

#####################################
# GIN-forkメタデータ取得API実装する #
#####################################

#######################################
# PKGを使ってRO-Crate化処理を実装する #
#######################################

## 2. メタデータを検証する

メタデータの検証と検証結果の生成を行う検証サービスに、メタデータの検証を依頼します。

In [ ]:
import requests
import os
import traceback

try:
    ###############################################
    # DG-Coreサーバーの仕様が固まり次第、更新する #
    ###############################################
    #req_response = requests.post("https://example.com/**/validate?entityIds=a&entityIds=b", json=crate_json)
    req_response = requests.get('https://api.github.com/events')
    #result = req_response.json
    req_response.status_code = 200
    result = {
        "request_id": "fffff",
        "request": {
            "roCrate": {
                "entity1" : "a"
            },
            "entityIds": ["a", "b"]
        },
        "results": {
            "result": "fffff"
        }
    }
except:
    print('検証サービスに接続できません。')
    print(traceback.format_exc())
else:
    if req_response.status_code == 200:
        os.chdir(os.environ['HOME'])
        from WORKFLOWS.utils.tmp_validation import tmp_validation
        request_id = result['request_id']
        tmp_validation.save_request_id(request_id)
        print('正常に検証が依頼されました。次の処理にお進みください。\nリクエストID：' + request_id)
    elif req_response.status_code == 400:
        print('BadRequest')
        print(result['message'])
    elif req_response.status_code == 500:
        print('InternalServerError')
        print(result['message'])

## 3. 検証結果を確認する

検証サービスから検証結果を取得し、結果を出力します。

In [ ]:
import os
import json
import requests
import time
from IPython.display import display, HTML, clear_output
from WORKFLOWS.utils.tmp_validation import tmp_validation

request_id = tmp_validation.get_request_id()
counter = 5

while counter >0:
    try:
        ###############################################
        # DG-Coreサーバーの仕様が固まり次第、更新する #
        ###############################################
        #result_response = requests.get("https://example.com/*/" + request_id)
        result_response = requests.get('https://api.github.com/events')
        #result = result_response.json
        result = {
        "status": "COMPLETE",
        "request": {
            "roCrate": {
                "entity1" : "a"
            },
            "entityIds": ["a", "b"]
        },
        "results": {
            "result": "13:40"
        }
    }
        counter -= 1
    except:
        print('検証サービスに接続できません。')
        break
    else:
        if result_response.status_code == 200:
            status = result['status']
            if status == 'UNKNOWN':
                print('リクエストID：' + request_id + 'は正常にリクエストできていません。')
                break
            elif any([status == 'QUEUED', status == 'RUNNING']):
                clear_output()
                print('リクエストID：' + request_id + 'は検証完了していません。時間をおいて再度確認します。')
                time.sleep(5)
                continue
            elif status == 'COMPLETE':
                tmp_validation.save_verification_results(result)
                result_file_path = tmp_validation.fetch_validation_results_file_path()
                print('正常に検証結果を取得しました。以下のリンクから検証結果を確認し、次にお進みください。')
                display(HTML('<a href=../../../../edit/' + result_file_path + '>検証結果を確認する</a>'))
                break
            elif any([status == 'FAILED', status == 'EXECUTOR_ERROR']):
                print('正常に検証できませんでした。')
                break
            elif status == 'CANCELING':
                print('リクエストID：' + request_id + 'は現在キャンセル中です。')
                break
            elif status == 'CANCELED':
                print('リクエストID：' + request_id + 'はキャンセルされました。')
                break

        elif result_response.status_code == 400:
            print('BadRequest')
            print(result['message'])
            break
        elif result_response.status_code == 500:
            print('InternalServerError')
            print(result['message'])
            break
                      
else:
    clear_output()
    print('検証に時間がかかっています。時間をおいて再度このセルを実行してください。')
    raise Exception

## 4. 研究リポジトリに同期する

このタスクの実行結果を研究リポジトリに同期します。  
検証結果を同期するか破棄するかは、「4.1. 検証結果を研究リポジトリに同期するか選択する」で選択できます。

### 4.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する

In [ ]:
import panel as pn

pn.extension()
column = pn.Column()

def save_selection_result(event):
    global need_sync
    done_button.button_type = "success"
    done_button.name = "選択完了しました。次の処理にお進みください。"
    need_sync = True if select.value == 1 else False

select = pn.widgets.Select(name='検証結果を同期するか破棄するかを選択した後、完了ボタンをクリックしてください。', options={'同期する': 1, '同期せずに破棄する':2})
done_button = pn.widgets.Button(name= "選択を完了する", button_type= "primary")
done_button.on_click(save_selection_result)
column.append(select)
column.append(done_button)
column

### 4.2. 研究リポジトリに同期する

研究リポジトリにこのタスクの実行結果を同期します。  
「4.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する」で同期せずに破棄するを選択した場合は、検証結果は同期されずこのファイルの実行結果のみが同期されます。

In [ ]:
import os
from IPython.display import display, Javascript
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])
from WORKFLOWS.utils.tmp_validation import tmp_validation

tmp_validation.operate_validation_results(need_sync)
git_path = ['/home/jovyan/WORKFLOWS/FLOW/02_experimental_phase/base_validate_metadata.ipynb']
if need_sync == True:
    validation_result_path = tmp_validation.fetch_validation_result_path()
    git_path.append(validation_result_path)

display(Javascript('IPython.notebook.save_checkpoint();'))
utils.syncs_with_repo(git_path=git_path, gitannex_path=None, gitannex_files=None, message='メタデータ検証')

## 5. 研究ワークフロー機能トップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして研究ワークフロー機能トップページに戻ってください。

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../base_FLOW.ipynb'>研究ワークフロー機能トップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))